## Model Selection Rationale: Siamese Preference Transformer
The core task is a Pairwise Comparison Problem where the goal is to predict the preference between two financial time-series sequences (A and B). So I adopted a Siamese Preference Network architecture, which is ideally suited for learning the similarity or relational metrics between two distinct inputs.

Initial experiments with only Base Features showed low accuracy. To enhance the model's generalization ability, a systematic feature engineering process was executed. I used various combinations of features using Profitability/Duration (Log_Return, Proxy_Return, Time_in_Position), Momentum/Trend (MACD, RSI), and Volatility (ATR, Hist_Vol).

## Model Settings
0. Base
1. 0 + log return + proxy return
2. 1 + time in position
3. 2 + Momentum/Trend (MACD,MACD Signal,RSI)
4. 3 + Volatility (ATR,Historical Volatility)
5. 1 + Volatility (ATR,Historical Volatility)
6. 1 + N_LAYERS (4->6)
7. 6 + MODEL_DIM (256->512)
8. 6 + MODEL_DIM (256) + N_LAYERS (6->8) 

## Best Setting with Public Accuracy = 0.81

1. Features: Open, High, Low, Close, Volume, Position, Log Return, Proxy Return
2. MODEL_DIM = 256
3. N_LAYERS = 8
4. N_HEADS = 4
5. DROPOUT_RATE = 0.1
6. LEARNING_RATE = 1e-4
7. BATCH_SIZE = 32
8. EPOCHS = 50 with (PATIENCE = 10 Early Stopping by val loss)

Model 8 showed best public accuracy = 0.81, while Model 6 with the lowest val loss showed public accuracy = 0.77. This validates the principle that for this financial time-series problem, generalization capability (Decision Boundary) is more critical than the model's confidence level (Loss).

## Import packages

In [ ]:
import os
from collections import Counter
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(DEVICE) 

## Initial Hyperparameters

In [ ]:
INPUT_DIM = 4
MODEL_DIM = 256
N_HEADS = 4
N_LAYERS = 4
DROPOUT_RATE = 0.1
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
RANDOM_STATE=42
TEST_SIZE=0.2
NUM_EPOCHS=50
PATIENCE=10

## PrepareDataset for train/test

In [ ]:
class PrepareDataset(Dataset):
    def __init__(self, df, base_path):
        self.df = df
        self.base_path = base_path
        
        # Check whether "label" column exists
        self.has_labels = 'label' in df.columns
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        pair_id = row['id']
        
        try:
            # Load raw sequence files
            file_a = f"{self.base_path}sample_{pair_id}_a.csv"
            file_b = f"{self.base_path}sample_{pair_id}_b.csv"
            
            data_a = pd.read_csv(file_a, sep='\s+', header=None, on_bad_lines='skip')
            data_b = pd.read_csv(file_b, sep='\s+', header=None, on_bad_lines='skip')

            # CRITICAL: Apply preprocessing pipeline
            data_a_processed = preprocess_sequence(data_a.copy())
            data_b_processed = preprocess_sequence(data_b.copy())
            
            seq_a = torch.tensor(data_a_processed, dtype=torch.float32)
            seq_b = torch.tensor(data_b_processed, dtype=torch.float32)
            
        except Exception as e:
            print(f"Error loading sequences for ID {pair_id}: {e}")
            # When failed to load sequence, return empty tensors so collate_fn can handle it
            return torch.empty(0), torch.empty(0), -1 

        # Handle label existence (train/test mode)
        if self.has_labels:
            label = torch.tensor([row['label']], dtype=torch.float32)
            # Training mode: return (A, B, label)
            return seq_a, seq_b, label
        else:
            # Test mode: return (A, B, id)
            return seq_a, seq_b, pair_id

## Sequence Encoder and Siamese Network for Transformer Set-up

In [ ]:
class SequenceEncoder(nn.Module):
    def __init__(self, input_dim=INPUT_DIM, model_dim=MODEL_DIM, n_heads=N_HEADS, n_layers=N_LAYERS):
        super().__init__()
        self.model_dim = MODEL_DIM
        
        # 1. Map input dimension to model_dim
        self.input_layer = nn.Linear(input_dim, model_dim)
        
        # 2. Define the Transformer Encoder Layer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim, 
            nhead=n_heads, 
            dim_feedforward=model_dim * 4,
            # Assuming DROPOUT_RATE is defined
            dropout=DROPOUT_RATE, 
            batch_first=True # Order is (Batch, Seq, Feature)
        )
        
        # 3. Define the Transformer Encoder Stack
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
    def forward(self, x):
        # x shape: (Batch Size, Sequence Length, Input Dim)
        
        # Map input dimension
        x = F.normalize(x, dim=-1)
        x = self.input_layer(x)
        
        # Pass through the Transformer Encoder
        output = self.transformer_encoder(x)
        
        # Final Embedding: A vector representing the entire sequence (Pooling)
        # 1. Mean Pooling: Take the average along the sequence length dimension
        embedding = output.mean(dim=1) 
        # embedding shape: (Batch Size, Model Dim)
        
        return embedding

class SiamesePreferenceModel(nn.Module):
    def __init__(self, input_dim=INPUT_DIM, model_dim=MODEL_DIM):
        super().__init__()
        
        # Define the identical encoder with shared weights
        self.encoder = SequenceEncoder(input_dim, model_dim)
        
        # Comparison Head (Receives z_A, z_B, and |z_A - z_B| as input)
        # Input dimension: 3 * model_dim
        head_input_dim = 3 * model_dim 
        
        self.classifier = nn.Sequential(
            nn.Linear(head_input_dim, model_dim),
            nn.ReLU(),
            nn.Dropout(DROPOUT_RATE),
            nn.Linear(model_dim, model_dim // 2),
            nn.ReLU(),
            nn.Linear(model_dim // 2, 1) # Final output is 1 (probability)
        )
        
    def forward(self, seq_a, seq_b):
        # Pass sequences A and B through the encoder to generate embeddings (shared weights)
        z_a = self.encoder(seq_a)
        z_b = self.encoder(seq_b)
        
        # Construct the feature vector for comparison
        diff = torch.abs(z_a - z_b) # Absolute difference
        
        # Concatenation: [z_a, z_b, |z_a - z_b|]
        comparison_vector = torch.cat((z_a, z_b, diff), dim=1)
        
        # Pass through the classifier and apply Sigmoid
        logits = self.classifier(comparison_vector)
        prediction = torch.sigmoid(logits)
        
        # prediction shape: (Batch Size, 1)
        return prediction, z_a, z_b

## Early Stopper

In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0, path='model_best.pth'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self._save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self._save_checkpoint(val_loss, model)
            self.counter = 0

    def _save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

## Model Trainer

In [ ]:
def train_model(model, train_dataloader, val_dataloader, num_epochs=50, patience=10, model_save_path='model_best.pth'):
    criterion = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
    model.to(DEVICE)
    
    # Early Stopping Init
    early_stopping = EarlyStopping(patience=patience, verbose=True, path=model_save_path)
    
    print("--- Start Training ---")
    print(f"Layers: {N_LAYERS} | Model Dim: {MODEL_DIM} | Fixed LR: {LEARNING_RATE} | Save Path: {model_save_path}")
    
    for epoch in range(num_epochs):
        model.train() 
        total_train_loss = 0
        correct_train_predictions = 0
        total_train_samples = 0
        
        # 1. Training Step
        for seq_a, seq_b, labels in train_dataloader:
            if len(labels) == 0:
                continue
            
            seq_a, seq_b, labels = seq_a.to(DEVICE), seq_b.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            predictions, _, _ = model(seq_a, seq_b)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * labels.size(0)
            predicted_classes = (predictions >= 0.5).float()
            correct_train_predictions += (predicted_classes == labels).sum().item()
            total_train_samples += labels.size(0)

        # calculate train_accuracy and loss
        avg_train_loss = total_train_loss / total_train_samples if total_train_samples > 0 else 0
        train_accuracy = correct_train_predictions / total_train_samples if total_train_samples > 0 else 0

        # 2. Validation Step
        val_loss, val_accuracy = validate_model(model, val_dataloader, criterion)
        
        # 3. Print output and check Early Stopping
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_accuracy:.4f}")
        
        # Early Stopping by val_loss
        early_stopping(val_loss, model)
        
        if early_stopping.early_stop:
            print("--- Early Stopping: Stop training, no more decrese in validation loss ---")
            break

    # 4. Load the best model setting
    model.load_state_dict(torch.load(model_save_path))
    print(f"--- Training Complete ('{model_save_path}' Loaded) ---")
    return model

## Validation Model

In [ ]:
def validate_model(model, dataloader, criterion):
    model.eval() 
    total_val_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad():
        for seq_a, seq_b, labels in dataloader:
            if len(labels) == 0:
                continue
            
            seq_a, seq_b, labels = seq_a.to(DEVICE), seq_b.to(DEVICE), labels.to(DEVICE)
            
            predictions, _, _ = model(seq_a, seq_b)
            loss = criterion(predictions, labels)
            
            total_val_loss += loss.item() * labels.size(0)
            
            predicted_classes = (predictions >= 0.5).float()
            correct_predictions += (predicted_classes == labels).sum().item()
            total_samples += labels.size(0)
            
    avg_loss = total_val_loss / total_samples if total_samples > 0 else 0
    accuracy = correct_predictions / total_samples if total_samples > 0 else 0
    model.train() 
    return avg_loss, accuracy

from torch.utils.data import Dataset

class PairSequenceDataset(Dataset):
    """
    Example dataset for (seq_a, seq_b, label) pairs using selected features.
    raw_pairs: list of (df_a, df_b, label)
    """

    def __init__(self, raw_pairs, selected_features):
        self.raw_pairs = raw_pairs
        self.selected_features = selected_features

    def __len__(self):
        return len(self.raw_pairs)

    def __getitem__(self, idx):
        df_a, df_b, label = self.raw_pairs[idx]

        seq_a = preprocess_sequence(df_a, self.selected_features)
        seq_b = preprocess_sequence(df_b, self.selected_features)

        # You may need to convert to torch tensors in collate_fn or here
        return seq_a, seq_b, label


In [ ]:
train_df = pd.read_csv('train.csv')

pair_ids = train_df['id'].unique()
train_ids, val_ids = train_test_split(
    pair_ids, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE,
    shuffle=True
)

In [ ]:
train_df_80 = train_df[train_df['id'].isin(train_ids)].reset_index(drop=True)
val_df_20 = train_df[train_df['id'].isin(val_ids)].reset_index(drop=True)

print(f"Total Sample: {len(train_df)}")
print(f"Training Sample (80%): {len(train_df_80)}")
print(f"Validation Sample (20%): {len(val_df_20)}")

## collate_fn() for batches

In [ ]:
def collate_fn(batch):
    batch = [item for item in batch if item[2] != -1]
    if not batch:
        return torch.empty(0), torch.empty(0), [] 

    # 1. Split Sequence and Label/ID
    seqs_a = [item[0] for item in batch]
    seqs_b = [item[1] for item in batch]
    labels_or_ids = [item[2] for item in batch]

    # 2. Sequence Padding
    padded_seqs_a = pad_sequence(seqs_a, batch_first=True, padding_value=0.0)
    padded_seqs_b = pad_sequence(seqs_b, batch_first=True, padding_value=0.0)
    
    # 3. Label/ID
    if torch.is_tensor(labels_or_ids[0]) and labels_or_ids[0].dtype == torch.float32:
        labels_batch = torch.cat(labels_or_ids, dim=0).view(-1, 1)
        return padded_seqs_a, padded_seqs_b, labels_batch
    
    # 3.2. Test Data (If item[2] is scalar ID)
    else:
        return padded_seqs_a, padded_seqs_b, labels_or_ids

## Preprocess_sequence

In [ ]:
def preprocess_sequence(df: pd.DataFrame) -> np.ndarray:
    # 1. OHLCV + Position
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Position']

    # 2. Remove NaN
    df = df.dropna().reset_index(drop=True)
    
    # 3. Z-Score Standardisation
    features_to_scale = ['Open', 'High', 'Low', 'Close', 'Volume'] 
    
    for feature in features_to_scale:
        mean = df[feature].mean()
        std = df[feature].std()
        
        if std != 0:
            df.loc[:, feature] = (df[feature] - mean) / std
        else:
            df.loc[:, feature] = 0 

    # 4. Final Features
    final_features = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Position']].values
    
    return final_features.astype(np.float32)

## Checking Class Imbalance

In [ ]:
def check_class_distribution(train_df):
    label_counts = train_df['label'].value_counts()
    total_samples = len(train_df)

    print(f"Total sample: {total_samples}")
    print("-" * 30)

    distribution_data = []
    for label, count in label_counts.items():
        percentage = (count / total_samples) * 100
        meaning = "A wins (label 0)" if label == 0 else "B wins (label 1)"
        distribution_data.append([label, meaning, count, f"{percentage:.2f}%"])

    distribution_df = pd.DataFrame(distribution_data, 
                                   columns=['label', 'meaning', 'count', 'ratio'])
    
    print(distribution_df.to_markdown(index=False))

    if len(label_counts) == 2:
        ratio = label_counts.max() / label_counts.min()
        print("-" * 30)

check_class_distribution(train_df)

## Model 0: Train basic model with Initial Parameters and set-ups

Epoch 14/50 | Train Loss: 0.6554 | Train Acc: 0.6145 | Val Loss: 0.6575 | Val Acc: 0.5952

In [ ]:
train_dataset = PrepareDataset(train_df_80, base_path='./train/')
val_dataset = PrepareDataset(val_df_20, base_path='./train/')

train_dataloader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_fn
)
val_dataloader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    collate_fn=collate_fn
)

INPUT_DIM = 6
MODEL_DIM = 256
model = SiamesePreferenceModel(input_dim=INPUT_DIM, model_dim=MODEL_DIM)
model.to(DEVICE) 

trained_model_0 = train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    num_epochs=NUM_EPOCHS, 
    patience=PATIENCE,
    model_save_path="trained_model_0.pth"
)

## Model 1: Log_Return and Proxy

Train Loss: 0.4665 | Train Acc: 0.7711 | Val Loss: 0.4987 | Val Acc: 0.7619

In [ ]:
def preprocess_sequence(df: pd.DataFrame) -> np.ndarray:
    # 1. OHLCV + Position
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Position']

    # 2. Log Return
    df.loc[:, 'Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # 3. Proxy Return
    df.loc[:, 'Proxy_Return'] = df['Position'] * df['Log_Return']

    # 4. Remove NaN
    df = df.dropna().reset_index(drop=True)
    
    # 5. Z-Score Standardisation
    features_to_scale = ['Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume'] 
    
    for feature in features_to_scale:
        mean = df[feature].mean()
        std = df[feature].std()
        
        if std != 0:
            df.loc[:, feature] = (df[feature] - mean) / std
        else:
            df.loc[:, feature] = 0 

    # 6. Final Features
    final_features = df[['Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume', 'Position']].values
    
    return final_features.astype(np.float32)

In [ ]:
INPUT_DIM = 8
MODEL_DIM = 256
model = SiamesePreferenceModel(input_dim=INPUT_DIM, model_dim=MODEL_DIM)
model.to(DEVICE) 

trained_model_1 = train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    num_epochs=NUM_EPOCHS, 
    patience=PATIENCE,
    model_save_path="trained_model_1.pth"
)

## Model 2: Add Time-in-Position Feature

Epoch 24/50 | Train Loss: 0.4069 | Train Acc: 0.8253 | Val Loss: 0.5338 | Val Acc: 0.7619

In [ ]:
def preprocess_sequence(df: pd.DataFrame) -> np.ndarray:
    # 1. OHLCV + Position
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Position']

    # 2. Log Return
    df.loc[:, 'Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # 3. Proxy Return
    df.loc[:, 'Proxy_Return'] = df['Position'] * df['Log_Return']

    # 4. Time-in-Position
    df.loc[:, 'Time_in_Position'] = 0
    current_position_count = 0
    
    for i in range(len(df)):
        current_Position = df.iloc[i]['Position']
        
        if current_Position == 1:
            current_position_count += 1
        else:
            current_position_count = 0
        
        df.iloc[i, df.columns.get_loc('Time_in_Position')] = current_position_count

    # 5. Remove NaN
    df = df.dropna().reset_index(drop=True)
    
    # 6. Z-Score Standardisation
    features_to_scale = ['Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume', 'Time_in_Position'] 
    
    for feature in features_to_scale:
        mean = df[feature].mean()
        std = df[feature].std()
        
        if std != 0:
            df.loc[:, feature] = (df[feature] - mean) / std
        else:
            df.loc[:, feature] = 0 

    # 7. Final Features
    final_features = df[['Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume', 'Position', 'Time_in_Position']].values
    
    return final_features.astype(np.float32)

In [ ]:
INPUT_DIM = 9
MODEL_DIM = 256
model = SiamesePreferenceModel(input_dim=INPUT_DIM, model_dim=MODEL_DIM)
model.to(DEVICE) 

trained_model_2 = train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    num_epochs=NUM_EPOCHS, 
    patience=PATIENCE,
    model_save_path="trained_model_2.pth"
)

## Model 3: add MACD, MACD_Signal, RSI

Epoch 16/50 | Train Loss: 0.6743 | Train Acc: 0.6084 | Val Loss: 0.6629 | Val Acc: 0.5952

In [ ]:
def preprocess_sequence(df: pd.DataFrame) -> np.ndarray:
    # 1. OHLCV + Position
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Position']

    # 2. Log Return
    df.loc[:, 'Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # 3. Proxy Return
    df.loc[:, 'Proxy_Return'] = df['Position'] * df['Log_Return']

    # 4. Time-in-Position
    df.loc[:, 'Time_in_Position'] = 0
    current_position_count = 0
    
    time_in_position_column_index = df.columns.get_loc('Time_in_Position')
    Position_column_index = df.columns.get_loc('Position')

    for i in range(len(df)):
        current_Position = df.iloc[i, Position_column_index]
        
        if current_Position == 1:
            current_position_count += 1
        else:
            current_position_count = 0
            
        df.iloc[i, time_in_position_column_index] = current_position_count
        
    # 5. MACD & RSI
    
    # MACD
    exp1 = df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = df['Close'].ewm(span=26, adjust=False).mean()
    df.loc[:, 'MACD'] = exp1 - exp2
    df.loc[:, 'MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean() # Signal Line

    # RSI
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).ewm(span=14, adjust=False).mean()
    loss = (-delta.where(delta < 0, 0)).ewm(span=14, adjust=False).mean()
    # rs가 0으로 나뉘는 것을 방지
    rs = np.divide(gain, loss.replace(0, np.nan)) 
    df.loc[:, 'RSI'] = 100 - (100 / (1 + rs))
    # ----------------------------------------------------

    # 6. Remove NaN
    df = df.dropna().reset_index(drop=True)
    
    # 7. Z-Score Standardisation
    features_to_scale = [
        'Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume', 'Time_in_Position', 
        'MACD', 'MACD_Signal', 'RSI'
    ]
    
    for feature in features_to_scale:
        mean = df[feature].mean()
        std = df[feature].std()
        
        if std != 0:
            df.loc[:, feature] = (df[feature] - mean) / std
        else:
            df.loc[:, feature] = 0

    # 8. Final Features
    final_features = df[[
        'Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume', 'Position', 'Time_in_Position',
        'MACD', 'MACD_Signal', 'RSI'
    ]].values
    
    return final_features.astype(np.float32)

In [ ]:
INPUT_DIM = 12
MODEL_DIM = 256
model = SiamesePreferenceModel(input_dim=INPUT_DIM, model_dim=MODEL_DIM)
model.to(DEVICE) 

trained_model_3 = train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    num_epochs=NUM_EPOCHS, 
    patience=PATIENCE,
    model_save_path="trained_model_3.pth"
)

## Model 4: Add ATR, Historical Volatility

Train Loss: 0.5668 | Train Acc: 0.7771 | Val Loss: 0.6097 | Val Acc: 0.6667

In [ ]:
def preprocess_sequence(df: pd.DataFrame) -> np.ndarray:
    # 1. OHLCV + Position
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Position']

    # 2. Log Return
    df.loc[:, 'Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # 3. Proxy Return
    df.loc[:, 'Proxy_Return'] = df['Position'] * df['Log_Return']

    # 4. Time-in-Position
    df.loc[:, 'Time_in_Position'] = 0
    current_position_count = 0
    
    time_in_position_column_index = df.columns.get_loc('Time_in_Position')
    Position_column_index = df.columns.get_loc('Position')

    for i in range(len(df)):
        current_Position = df.iloc[i, Position_column_index]
        if current_Position == 1:
            current_position_count += 1
        else:
            current_position_count = 0
        df.iloc[i, time_in_position_column_index] = current_position_count

    # 5. MACD & RSI
    exp1 = df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = df['Close'].ewm(span=26, adjust=False).mean()
    df.loc[:, 'MACD'] = exp1 - exp2
    df.loc[:, 'MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean() 

    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).ewm(span=14, adjust=False).mean()
    loss = (-delta.where(delta < 0, 0)).ewm(span=14, adjust=False).mean()
    rs = np.divide(gain, loss.replace(0, np.nan)) 
    df.loc[:, 'RSI'] = 100 - (100 / (1 + rs))

    # 6. ATR & Historical Volatility
    
    # True Range (TR)
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift(1))
    low_close = np.abs(df['Low'] - df['Close'].shift(1))
    # TR: max(H-L, |H-C_prev|, |L-C_prev|)
    df.loc[:, 'TR'] = high_low.combine(high_close, max).combine(low_close, max)

    # Average True Range (ATR) - 14-period EMA
    df.loc[:, 'ATR'] = df['TR'].ewm(span=14, adjust=False).mean()
    
    # Historical Volatility (HV) - 14-period Rolling Std Dev of Log Returns
    df.loc[:, 'Historical_Volatility'] = df['Log_Return'].rolling(window=14).std()

    # 7. Remove NaN
    df = df.dropna().reset_index(drop=True)
    
    # 8. Z-Score Standardisation 
    features_to_scale = [
        'Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume', 'Time_in_Position', 
        'MACD', 'MACD_Signal', 'RSI', 'ATR', 'Historical_Volatility' 
    ]
    
    for feature in features_to_scale:
        mean = df[feature].mean()
        std = df[feature].std()
        if std != 0:
            df.loc[:, feature] = (df[feature] - mean) / std
        else:
            df.loc[:, feature] = 0

    # 9. Final Features
    final_features = df[[
        'Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume', 'Position', 'Time_in_Position',
        'MACD', 'MACD_Signal', 'RSI', 'ATR', 'Historical_Volatility' 
    ]].values
    
    return final_features.astype(np.float32)


In [ ]:
INPUT_DIM = 14
MODEL_DIM = 256
model = SiamesePreferenceModel(input_dim=INPUT_DIM, model_dim=MODEL_DIM)
model.to(DEVICE) 

trained_model_4 = train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    num_epochs=NUM_EPOCHS, 
    patience=PATIENCE,
    model_save_path="trained_model_4.pth"
)

## Model 5: Exclude MACD & RSI, Add ATR, Historical Volatility

Train Loss: 0.6885 | Train Acc: 0.5964 | Val Loss: 0.6905 | Val Acc: 0.5238

In [ ]:
def preprocess_sequence(df: pd.DataFrame) -> np.ndarray:
    # 1. OHLCV + Position
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Position']

    # 2. Log Return
    df.loc[:, 'Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # 3. Proxy Return
    df.loc[:, 'Proxy_Return'] = df['Position'] * df['Log_Return']

    # 4. Time-in-Position
    df.loc[:, 'Time_in_Position'] = 0
    current_position_count = 0
    
    time_in_position_column_index = df.columns.get_loc('Time_in_Position')
    Position_column_index = df.columns.get_loc('Position')

    for i in range(len(df)):
        current_Position = df.iloc[i, Position_column_index]
        if current_Position == 1:
            current_position_count += 1
        else:
            current_position_count = 0
        df.iloc[i, time_in_position_column_index] = current_position_count

    # 6. ATR & Historical Volatility
    
    # True Range (TR)
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift(1))
    low_close = np.abs(df['Low'] - df['Close'].shift(1))
    # TR: max(H-L, |H-C_prev|, |L-C_prev|)
    df.loc[:, 'TR'] = high_low.combine(high_close, max).combine(low_close, max)

    # Average True Range (ATR) - 14-period EMA
    df.loc[:, 'ATR'] = df['TR'].ewm(span=14, adjust=False).mean()
    
    # Historical Volatility (HV) - 14-period Rolling Std Dev of Log Returns
    df.loc[:, 'Historical_Volatility'] = df['Log_Return'].rolling(window=14).std()

    # 7. Remove NaN
    df = df.dropna().reset_index(drop=True)
    
    # 8. Z-Score Standardisation 
    features_to_scale = [
        'Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume', 'Time_in_Position', 
        'ATR', 'Historical_Volatility' 
    ]
    
    for feature in features_to_scale:
        mean = df[feature].mean()
        std = df[feature].std()
        if std != 0:
            df.loc[:, feature] = (df[feature] - mean) / std
        else:
            df.loc[:, feature] = 0

    # 9. Final Features
    final_features = df[[
        'Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume', 'Position', 'Time_in_Position',
        'ATR', 'Historical_Volatility' 
    ]].values
    
    return final_features.astype(np.float32)


In [ ]:
INPUT_DIM = 11
MODEL_DIM = 256
model = SiamesePreferenceModel(input_dim=INPUT_DIM, model_dim=MODEL_DIM)
model.to(DEVICE) 

trained_model_5 = train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    num_epochs=NUM_EPOCHS, 
    patience=PATIENCE,
    model_save_path="trained_model_5.pth"
)

## Model 6: Features = Base + Log Return + Proxy, NUM_LAYERS = 4 -> 6

Train Loss: 0.3107 | Train Acc: 0.8795 | Val Loss: 0.4977 | Val Acc: 0.7381

In [ ]:
def preprocess_sequence(df: pd.DataFrame) -> np.ndarray:
    # 1. OHLCV + Position
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Position']

    # 2. Log Return
    df.loc[:, 'Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # 3. Proxy Return
    df.loc[:, 'Proxy_Return'] = df['Position'] * df['Log_Return']

    # 4. Remove NaN
    df = df.dropna().reset_index(drop=True)
    
    # 5. Z-Score Standardisation
    features_to_scale = ['Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume'] 
    
    for feature in features_to_scale:
        mean = df[feature].mean()
        std = df[feature].std()
        
        if std != 0:
            df.loc[:, feature] = (df[feature] - mean) / std
        else:
            df.loc[:, feature] = 0 

    # 6. Final Features
    final_features = df[['Open', 'High', 'Low', 'Close', 'Log_Return', 'Proxy_Return', 'Volume', 'Position']].values
    
    return final_features.astype(np.float32)

In [ ]:
N_LAYERS = 6
INPUT_DIM = 8
MODEL_DIM = 256
model = SiamesePreferenceModel(input_dim=INPUT_DIM, model_dim=MODEL_DIM)
model.to(DEVICE) 
trained_model_6 = train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    num_epochs=NUM_EPOCHS, 
    patience=PATIENCE,
    model_save_path="trained_model_6.pth"
)

## Model 7: MODEL_DIM = 256 -> 512

Train Loss: 0.6589 | Train Acc: 0.6506 | Val Loss: 0.6684 | Val Acc: 0.5714

In [ ]:
N_LAYERS = 6
INPUT_DIM = 8
MODEL_DIM = 512
model = SiamesePreferenceModel(input_dim=INPUT_DIM, model_dim=MODEL_DIM)
model.to(DEVICE) 
trained_model_7 = train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    num_epochs=NUM_EPOCHS, 
    patience=PATIENCE,
    model_save_path="trained_model_7.pth"
)

## Model 8: MODEL_DIM = 256, N_LAYERS = 8

Train Loss: 0.4262 | Train Acc: 0.8373 | Val Loss: 0.5296 | Val Acc: 0.7857

In [ ]:
N_LAYERS = 8
INPUT_DIM = 8
MODEL_DIM = 256
model = SiamesePreferenceModel(input_dim=INPUT_DIM, model_dim=MODEL_DIM)
model.to(DEVICE) 
trained_model_8 = train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    num_epochs=NUM_EPOCHS, 
    patience=PATIENCE,
    model_save_path="trained_model_8.pth"
)

In [ ]:
def predict_submission(model, dataloader, model_path):
    try:
        model.load_state_dict(torch.load(model_path, map_location=DEVICE))
        print(f"Loaded model weights from {model_path}")
    except FileNotFoundError:
        print(f"🚨 FATAL ERROR: Can't find '{model_path}'")
        return
    except Exception as e:
        print(f"🚨 ERROR loading model: {e}")
        return

    model.eval()
    model.to(DEVICE)
    
    predictions = []
    row_ids = []

    with torch.no_grad():
        for seq_a, seq_b, ids in dataloader:
            
            if len(ids) == 0:
                continue
                
            seq_a = seq_a.to(DEVICE)
            seq_b = seq_b.to(DEVICE)

            probabilities, _, _ = model(seq_a, seq_b)
            
            final_predictions = (probabilities > 0.5).int()
            
            predictions.extend(final_predictions.cpu().numpy().flatten())
            row_ids.extend(ids) 

    print(f"Total predictions: {len(predictions)}")
    
    submission_df = pd.DataFrame({
        'id': row_ids, 
        'pred': predictions
    })
    
    submission_df['id'] = submission_df['id'].astype(int)
    submission_df = submission_df.sort_values(by='id').reset_index(drop=True)
    
    submission_file_path = 'submission.csv'
    submission_df.to_csv(submission_file_path, index=False)
    print(f"File '{submission_file_path}' generated.")
    print(submission_df.head())

## Model 6: Public score 0.77

In [ ]:
MODEL_DIM = 256
model = SiamesePreferenceModel(input_dim=INPUT_DIM, model_dim=MODEL_DIM)
model.to(DEVICE) 
test_df = pd.read_csv('test.csv') 
test_dataset = PrepareDataset(test_df, base_path='./test/')
TEST_BATCH_SIZE = 32 
test_dataloader = DataLoader(
    test_dataset, 
    batch_size=TEST_BATCH_SIZE, 
    shuffle=False,
    collate_fn=collate_fn
)

predict_submission(model, test_dataloader, model_path='trained_model_6.pth')

## Model 8: Public Score 0.81

In [ ]:
N_LAYERS = 8
INPUT_DIM = 8
MODEL_DIM = 256
model = SiamesePreferenceModel(input_dim=INPUT_DIM, model_dim=MODEL_DIM)
model.to(DEVICE) 
test_df = pd.read_csv('test.csv') 
test_dataset = PrepareDataset(test_df, base_path='./test/')
TEST_BATCH_SIZE = 32 
test_dataloader = DataLoader(
    test_dataset, 
    batch_size=TEST_BATCH_SIZE, 
    shuffle=False,
    collate_fn=collate_fn
)

predict_submission(model, test_dataloader, model_path='trained_model_8.pth')